In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib as mpl

mpl.style.use("seaborn-v0_8-deep")
mpl.rcParams["figure.figsize"] = (14, 5)
mpl.rcParams["figure.dpi"] = 100

### Loading the datafile for cleaning and quick view

In [2]:
data = pd.read_csv('../data/raw/nigeria_telecom_transactions_messy_actual_cities.csv')

print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55000 entries, 0 to 54999
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Transaction ID       49431 non-null  object 
 1   Customer ID          49515 non-null  object 
 2   Transaction Date     49445 non-null  object 
 3   Operator Name        55000 non-null  object 
 4   Transaction Type     55000 non-null  object 
 5   Transaction Amount   49566 non-null  object 
 6   Customer Age         49602 non-null  float64
 7   Customer Gender      49534 non-null  object 
 8   Customer Location    49584 non-null  object 
 9   Service Plan         55000 non-null  object 
 10  Data Usage (MB)      49533 non-null  object 
 11  Call Duration (min)  49587 non-null  object 
 12  SMS Sent             49467 non-null  float64
 13  Internet Package     55000 non-null  object 
 14  Transaction Status   55000 non-null  object 
dtypes: float64(2), object(13)
memory usa

,Transaction ID,Customer ID,Transaction Date,Operator Name,Transaction Type,Transaction Amount,Customer Age,Customer Gender,Customer Location,Service Plan,Data Usage (MB),Call Duration (min),SMS Sent,Internet Package,Transaction Status
0,66d1c0da-4597-4458-b4db-b2e94fe9c69f,f5754858-fe7d-4cc9-a1ff-80ce6d64ca90,NaN,Airtel,Ar1tel,NaN,27.0,NaN,Minna,Postpaid,4826.08,214.72,51.0,Daily,Pending
1,21ca8795-8ad9-47bc-a16b-e4654697ee52,NaN,2023-10-29T02:48:20.227058,9mobile,Bill Payment,4804.1,39.0,Other,Kaduna,Postpaid,3440,234.3,13.0,Weekly,Failed
2,ea6478bf-6a8c-4d91-bd63-b4464ebc5ae0,3052574b-ae63-4e67-8d86-5f8418ef6f66,2024-04-03T02:22:20.359918,9mobile,Bill Payment,19303.03,NaN,male,Kaduna,Prepaid,NaN,0.9,0.0,Weekly,Completed
3,NaN,15ad55e7-2c9e-4a84-9b9d-9e7d7747a177,2024-02-07T00:22:30.827156,9mobil3,Bill Payment,4094,NaN,Male,Owerri,Prepaid,2250.0,$16.38,20.0,Daily,Completed
4,a1165572-b0f4-42e2-9890-eb16ef0b8741,a4adc3b5-2b36-4ba5-b7d9-1a103bfb247a,2008-10-31 21:12:40,Glo,Data Purchase,NaN,36.0,female,Sokoto,Prepaid,3307.97,$92.61,58.0,Weekly,Failed


In [3]:
# Copying the data to a new dataframe so as to have what to compare the data to after cleaning
df = data.copy()

### Checking each Feature for errors in entry and fixing the datatype if need be

**Reusable Number of Missing Values Function**

In [4]:
def missing_values(column_name):
    """
    This function calculates and returns the number of missing values present in a specified column of the DataFrame.
    """
    return (f"Number of Missing values in the {column_name} column: {df[column_name].isnull().sum()}")

In [5]:
# Getting the percentage of the missing values by column
df.isna().mean() * 100

Transaction ID         10.125455
Customer ID             9.972727
Transaction Date       10.100000
Operator Name           0.000000
Transaction Type        0.000000
Transaction Amount      9.880000
Customer Age            9.814545
Customer Gender         9.938182
Customer Location       9.847273
Service Plan            0.000000
Data Usage (MB)         9.940000
Call Duration (min)     9.841818
SMS Sent               10.060000
Internet Package        0.000000
Transaction Status      0.000000
dtype: float64

It appears there are missing values in about 10 columns but compared to the size of the dataset the missing data is only about 10% of the entire dataset.
The decision to eithr remove the missing values or impute them would be made later. 

**Transaction Date Column**

In [6]:
# Getting an understanding of the types of values in the column
df['Transaction Date'].sample(8)

7991     2024-06-12T09:33:44.833949
35231    2024-03-24T15:12:12.831947
23090    2024-07-19T00:18:45.604970
39449    2023-10-21T16:09:15.863185
15811                           NaN
37019           2011-11-27 11:10:47
46847    2023-11-19T01:39:03.463869
5451     2024-04-22T14:49:59.301919
Name: Transaction Date, dtype: object

In [7]:
# Converting the Transaction Date column to datetime and removing the nano-seconds from it
df['Transaction Date'] = (
                            pd.to_datetime
                            (
                                df['Transaction Date']
                                .str.replace('T', ' ', regex=False)
                                .str.split('.')
                                .str[0]
                                )
                        )


In [8]:
df['Transaction Date'].sample(5)

23293   2024-06-03 17:54:38
18219                   NaT
30737   2024-06-16 15:03:04
47185   1975-06-09 02:43:25
34173   2024-04-13 03:20:03
Name: Transaction Date, dtype: datetime64[ns]

In [9]:
# Confirming that the number of null values in 'Transaction Date' is equal to df['Transaction Date'] after convertion
num_nulls_original = data['Transaction Date'].isnull().sum()
num_nulls_modified = df['Transaction Date'].isnull().sum()
assert num_nulls_original == num_nulls_modified, "The number of null values in 'Transaction Date' must be the same before and after modifications."

# Confirming that all the values follow the same format
assert (df['Transaction Date'].dropna().dt.strftime('%Y-%m-%d %H:%M:%S') == df['Transaction Date'].dropna()).all(), "Not all non-NaT dates follow the '%Y-%m-%d %H:%M:%S' format"

**Operator Name Column**

- Getting the types of values in the column

In [10]:
# Getting the types of values in the column
df['Operator Name'].value_counts()

Operator Name
MTN        12560
Airtel     12340
9mobile    12322
Glo        12284
Ar1tel      1413
G1o         1397
9mobil3     1387
MNT         1297
Name: count, dtype: int64

- Correcting the spelling errors

In [11]:
correction_map = {
    'Ar1tel': 'Airtel',
    'G1o': 'Glo',
    '9mobil3': '9mobile',
    'MNT': 'MTN'
}

# Apply the corrections
df['Operator Name'] = df['Operator Name'].replace(correction_map).astype('category')

In [12]:
df['Operator Name'].value_counts()

Operator Name
MTN        13857
Airtel     13753
9mobile    13709
Glo        13681
Name: count, dtype: int64

**Transaction Type Column**

In [13]:
# Getting the types of values in the column
df['Transaction Type'].value_counts()

Transaction Type
Bill Payment        16548
Airtime Purchase    16536
Data Purchase       16455
9mobil3              1430
Ar1tel               1345
G1o                  1343
MNT                  1343
Name: count, dtype: int64

it seems there' an error in entry where operators names where included as collection type.
We would have to remove these rows(entries) since there's now way to know how to map them to actual transaction types

In [14]:
# Filtering out the rows with the wrong transaction type
df = df[~df['Transaction Type'].isin(['9mobil3', 'Ar1tel', 'G1o', 'MNT'])]
df['Transaction Type'] = df['Transaction Type'].astype('category')

**Transaction Amount**

In [15]:
# Checking to see the values in the column
df['Transaction Amount'].sample(10)

54771         3430
47393    $10216.23
53023    $17164.09
39437         3062
14781    $14073.93
4379        8626.2
10974       1906.4
20412        11196
47759    $14257.17
51841     18936.69
Name: Transaction Amount, dtype: object

We can see that some of the entries have $ signs in front and possibly whitespaces.
So we have to deal with those first before converting the column to float

In [16]:
df['Transaction Amount'] =(
                            df['Transaction Amount']
                            .str.strip()
                            .str.replace('$','')
                            .astype('float')
                        )

In [17]:
# Making sure we did not create more missing values when cleaning the column
assert (df['Transaction Amount'].isna().sum()) == 4867 , 'There are more missing values than expected'

**Customer Age**

In [18]:
# Checking the values in the column
df['Customer Age'].sample(10)

29338    61.0
38430    46.0
11649    56.0
4627     30.0
38733    26.0
43994    20.0
28071    53.0
4173     48.0
45956    60.0
47096    26.0
Name: Customer Age, dtype: float64

In [19]:
# Checking if their are outliers that might be errors
df['Customer Age'].describe()

count    44705.000000
mean        44.062141
std         15.297023
min         18.000000
25%         31.000000
50%         44.000000
75%         57.000000
max         70.000000
Name: Customer Age, dtype: float64

The Min, Max and Mean age are well within expected ranges so it appears there's no outlier or errors in age entry

**Customer Gender**

In [20]:
# Understanding the types of values in the column
df['Customer Gender'].value_counts()

Customer Gender
male      8979
Female    8978
Other     8965
female    8867
Male      8812
Name: count, dtype: int64

It appears that there are spelling inconsistencies with the gender column and also a gender type "Other". The "Other" type are probably people who didn't want to indicate their gender so we will leave as is and only correct the spelling inconsistency.

In [21]:
gender_map = {
    'Male': 'male',
    'Female':'female',
    'Other': 'other'
}
df['Customer Gender'] = df['Customer Gender'].replace(gender_map, regex=False).astype('category')

**Customer Location**

In [22]:
# checking all the state names to see if there are any spelling mistakes or unusal state names
df['Customer Location'].unique()

array(['Kaduna', 'Owerri', 'Sokoto', 'Jos', 'Ilorin', 'Kano', nan,
       'Gombe', 'Lagos', 'Onitsha', 'Minna', 'Calabar', 'Abeokuta', 'Uyo',
       'Asaba', 'Maiduguri', 'Makurdi', 'Warri', 'Oyo', 'Awka', 'Jalingo',
       'Aba', 'Port Harcourt', 'Enugu', 'Akure', 'Osogbo', 'Benin City',
       'Bauchi', 'Abuja', 'Zaria', 'Ibadan'], dtype=object)

It seems there are no errors in the customer location column apart from having missing data which would be taking care of later along with the other columns with missing data 

**Service Plan**

In [23]:
df['Service Plan'].value_counts()

Service Plan
Prepaid     22335
Postpaid    22285
G1o          1310
9mobil3      1233
Ar1tel       1199
MNT          1177
Name: count, dtype: int64

Just like the Transaction Type column, it seems there are errors in entry where operators names where included as service plan column.
We would have to remove these rows(entries) since there's now way to know how to map them to actual service plans

In [24]:
# Filter out rows where 'Service Plan' matches any of the excluded plans
df = df[~df['Service Plan'].isin (['G1o','9mobil3','Ar1tel','MNT'])]

# Convert 'Service Plan' to category type for efficiency
df['Service Plan'] = df['Service Plan'].astype('category')

**Data Usage (MB)**

In [25]:
# Viewing how the data looks like
df['Data Usage (MB)'].sample(15)

6421       3730.1
20902     3892.41
659          4087
43766    $3732.86
39297         NaN
49041     3871.91
26843        1687
49179        2078
37750     1678.12
46992     4790.85
38999     3461.09
54893      3329.9
20100     4383.63
51424        2470
20401    $1783.13
Name: Data Usage (MB), dtype: object

As can be viewed above some of the values have a dollar sign before them. Next step is to remove these dollar signs and convert the column to a float datatype.

In [26]:
# Getting the amount of nan values in the column before cleaning and conversion
# This helps in making sure that there are not some edge values we are not aware of.
# Because if we have more nan after conversion, 
# then that means there are edge values we are not aware of that were converted to nan

df['Data Usage (MB)'].isna().sum()

np.int64(4540)

In [27]:
df['Data Usage (MB)'] = (
                            df['Data Usage (MB)']
                            .str.replace('$','', regex=False)
                            .astype('float')
                        )

In [28]:
assert (df['Data Usage (MB)'].isna().sum() == 4540), 'The column has more missing values than expected'

**Call Duration (min)**

In [29]:
df['Call Duration (min)'].sample(15)

47000        NaN
2886       211.5
29313         13
52140        NaN
33535      240.0
54078     164.01
6055     $132.87
3257      272.13
28354    $276.08
8704       268.3
23452      131.2
19721     $88.04
1389      132.06
23369    $267.19
20228      $4.18
Name: Call Duration (min), dtype: object

The Call duration column has the same issues as Data Usage column as above. So we can Just reuse what was done above

In [30]:
df['Call Duration (min)'] = (
                            df['Call Duration (min)']
                            .str.replace('$','', regex=False)
                            .astype('float')
                        )

In [31]:
# Checking if their are outliers that might be errors
df['Call Duration (min)'].describe()

count    40238.000000
mean       149.906659
std         86.308030
min          0.000000
25%         75.812500
50%        149.425000
75%        224.735000
max        300.000000
Name: Call Duration (min), dtype: float64

**SMS Sent**

In [32]:
df['SMS Sent'].sample(15)

36182     NaN
47175     5.0
37238    83.0
1159     80.0
28881    60.0
32861    10.0
13966     NaN
47014    44.0
36851    29.0
19470     8.0
25396    51.0
21022     5.0
7866      NaN
17052     3.0
4982      9.0
Name: SMS Sent, dtype: float64

In [33]:
# Checking if their are outliers that might be errors
df['SMS Sent'].describe()

count    40135.000000
mean        49.537461
std         29.092979
min          0.000000
25%         24.000000
50%         49.000000
75%         75.000000
max        100.000000
Name: SMS Sent, dtype: float64

It appears the sms sent column is error free(No outliers and no error in entry) except for the missing values that would be dealt with later

**Internet Package**

In [34]:
df['Internet Package'].sample(15)

16765    Monthly
41730      Daily
7175     Monthly
16319    Monthly
1024      Weekly
23693    Monthly
21362     Weekly
41045     Weekly
44518    Monthly
18184    Monthly
4253      Weekly
41006      Daily
34961      Daily
28622     Weekly
37216      Daily
Name: Internet Package, dtype: object

In [35]:
df['Internet Package'].value_counts()

Internet Package
Weekly     13430
Daily      13322
Monthly    13282
G1o         1220
9mobil3     1165
Ar1tel      1139
MNT         1062
Name: count, dtype: int64

Just like the Transaction Type and Service Plan columns, it seems there are errors in entry where operators names where included as internet package column.
We would have to remove these rows(entries) since there's now way to know how to map them to actual Internet Packages.

In [36]:
# Filter out rows where 'Internet Package' matches any of the excluded plans
df = df[~df['Internet Package'].isin (['G1o','9mobil3','Ar1tel','MNT'])]

# Convert 'Internet Package' to category type for efficiency
df['Internet Package'] = df['Internet Package'].astype('category')

In [37]:
df['Internet Package'].value_counts()

Internet Package
Weekly     13430
Daily      13322
Monthly    13282
Name: count, dtype: int64

**Transaction Status**

In [38]:
df['Transaction Status'].sample(15)

29871       Failed
52934       Failed
35545       Failed
11030          MNT
53077      Pending
45681      Pending
37986       Ar1tel
4718       Pending
53059    Completed
54319       Failed
44996      Pending
28678      Pending
44991    Completed
6000       Pending
18029    Completed
Name: Transaction Status, dtype: object

In [39]:
df['Transaction Status'].value_counts()

Transaction Status
Completed    12046
Pending      12027
Failed       12008
Ar1tel        1012
9mobil3       1004
MNT            992
G1o            945
Name: count, dtype: int64

Transaction Status has the same issues as Internet package column and others. We would clean it the same way also

In [40]:
# Filter out rows where 'Transaction Status' matches any of the excluded plans
df = df[~df['Transaction Status'].isin (['G1o','9mobil3','Ar1tel','MNT'])]

# Convert 'Transaction Status' to category type for efficiency
df['Transaction Status'] = df['Transaction Status'].astype('category')

In [41]:
df['Transaction Status'].value_counts()

Transaction Status
Completed    12046
Pending      12027
Failed       12008
Name: count, dtype: int64

Initial Data cleaning completed. 
The initial cleaned csv will be saved to the interim data folder
And a final cleaning(Dealing with missing values etc) will be done on the datasheet

In [42]:
# Checking the dataframe before saving
print(df.info())
df.head()


<class 'pandas.core.frame.DataFrame'>
Index: 36081 entries, 1 to 54998
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Transaction ID       32466 non-null  object        
 1   Customer ID          32503 non-null  object        
 2   Transaction Date     32473 non-null  datetime64[ns]
 3   Operator Name        36081 non-null  category      
 4   Transaction Type     36081 non-null  category      
 5   Transaction Amount   32507 non-null  float64       
 6   Customer Age         32625 non-null  float64       
 7   Customer Gender      32501 non-null  category      
 8   Customer Location    32537 non-null  object        
 9   Service Plan         36081 non-null  category      
 10  Data Usage (MB)      32427 non-null  float64       
 11  Call Duration (min)  32548 non-null  float64       
 12  SMS Sent             32441 non-null  float64       
 13  Internet Package     36081 non-null 

,Transaction ID,Customer ID,Transaction Date,Operator Name,Transaction Type,Transaction Amount,Customer Age,Customer Gender,Customer Location,Service Plan,Data Usage (MB),Call Duration (min),SMS Sent,Internet Package,Transaction Status
1,21ca8795-8ad9-47bc-a16b-e4654697ee52,NaN,2023-10-29 02:48:20,9mobile,Bill Payment,4804.10,39.0,other,Kaduna,Postpaid,3440.00,234.30,13.0,Weekly,Failed
2,ea6478bf-6a8c-4d91-bd63-b4464ebc5ae0,3052574b-ae63-4e67-8d86-5f8418ef6f66,2024-04-03 02:22:20,9mobile,Bill Payment,19303.03,NaN,male,Kaduna,Prepaid,NaN,0.90,0.0,Weekly,Completed
3,NaN,15ad55e7-2c9e-4a84-9b9d-9e7d7747a177,2024-02-07 00:22:30,9mobile,Bill Payment,4094.00,NaN,male,Owerri,Prepaid,2250.00,16.38,20.0,Daily,Completed
4,a1165572-b0f4-42e2-9890-eb16ef0b8741,a4adc3b5-2b36-4ba5-b7d9-1a103bfb247a,2008-10-31 21:12:40,Glo,Data Purchase,NaN,36.0,female,Sokoto,Prepaid,3307.97,92.61,58.0,Weekly,Failed
7,07c57156-2d68-433b-92c2-8839db514bc0,NaN,NaT,Glo,Data Purchase,4499.60,49.0,male,Ilorin,Prepaid,3706.69,3.42,59.0,Daily,Pending


In [43]:
df.to_csv('../data/interim/1.0-ji-initial-data-cleaned', index=False)